In [1]:
import numpy as np
import parmed as pmd
from utils import base_units

# MosDef packages
import foyer
from foyer import Forcefield
#import hoomd
import mbuild as mb



In [2]:
# User inputs

In [3]:
def compound(smiles_string, remove_):
    
    comp_mb = mb.load(p3ht_smi, smiles = True) # mbuild compound
    comp_pmd = comp_mb.to_parmed() #parmed compound
    comp_mass = np.sum([atom.mass for atom in comp_pmd.atoms]) # Mass of the compound
    comp_mb.mass = comp_mass # Update mb compound's mass

    GAFF = foyer.forcefields.load_GAFF()
    comp_typed = GAFF.apply(comp_pmd, assert_dihedral_params=False)
    for atom_pmd, atom_mb in zip(comp_typed, comp_mb):
        atom_mb.name = "_{}".format(atom_pmd.type)
    
    return comp_mb

def build_box(compound, num_compounds, density):
    system_box = mb.packing.fill_box(compound, num_compounds, density)
    return system_box

In [57]:
'''
These steps below are essentially trying to recreate the process in planckton/placnkton/init.py
The goal is to start with a smiles string rather than an already typed .mol2 file
'''
# COMPOUND: 

# Set up compound
p3ht_smi = 'c1sccc1CCCCCC'
comp_mb = mb.load(p3ht_smi, smiles = True) # mbuild compound

# Find compound mass
comp_pmd = comp_mb.to_parmed() #parmed compound
comp_mass = np.sum([atom.mass for atom in comp_pmd.atoms]) # Mass of the compound
comp_mb.mass = comp_mass # Update mb compound's mass

# Type compound
GAFF = foyer.forcefields.load_GAFF()
comp_pmd_typed =  GAFF.apply(comp_mb, assert_dihedral_params=False) # parmed structure
#comp_mb_typed = mb.Compound() # create new mbuild compound
#comp_mb_typed.from_parmed(comp_pmd_typed) # Load from the typed parmed structure
for atom_pmd, atom_mb in zip(comp_pmd_typed, comp_mb):
    atom_mb.name = "{}".format(atom_pmd.type)  # Renaming mb comp atom names from atom types of parmed compound
    atom_mb.pos /= 10

In [42]:
atom_mb.pos

array([0.03469218, 0.07937259, 0.00884721])

In [58]:
[p.name for p in comp_mb.particles()]

['cc_r5',
 'ss',
 'cc_r5',
 'cc_r5',
 'cc_r5',
 'c3',
 'c3',
 'c3',
 'c3',
 'c3',
 'c3',
 'h4',
 'h4',
 'ha',
 'hc',
 'hc',
 'hc',
 'hc',
 'hc',
 'hc',
 'hc',
 'hc',
 'hc',
 'hc',
 'hc',
 'hc',
 'hc']

In [59]:
# CREATE A SIMULATION VOLUME (MBUILD BOX)
box = mb.packing.fill_box(compound = comp_mb, n_compounds=2, box=mb.Box([1,1,1]), edge=0.02, fix_orientation=True)

/home/chris/cme/mbuild/mbuild/compound.py:2443: UserWarning: Guessing that "<cc_r5 pos=( 0.0005, 0.0116,-0.0000), 0 bonds, id: 139869009703376>" is element: "C"
  atom, element))
/home/chris/cme/mbuild/mbuild/compound.py:2443: UserWarning: Guessing that "<ss pos=(-0.0112,-0.0008,-0.0001), 0 bonds, id: 139869009703440>" is element: "S"
  atom, element))
/home/chris/cme/mbuild/mbuild/compound.py:2443: UserWarning: Guessing that "<c3 pos=( 0.0259, 0.0154, 0.0000), 0 bonds, id: 139869009703568>" is element: "C"
  atom, element))
/home/chris/cme/mbuild/mbuild/compound.py:2443: UserWarning: Guessing that "<h4 pos=(-0.0029, 0.0219,-0.0001), 0 bonds, id: 139869007373264>" is element: "H"
  atom, element))
/home/chris/cme/mbuild/mbuild/compound.py:2443: UserWarning: Guessing that "<ha pos=( 0.0227,-0.0134, 0.0001), 0 bonds, id: 139869007375888>" is element: "H"
  atom, element))
/home/chris/cme/mbuild/mbuild/compound.py:2443: UserWarning: Guessing that "<hc pos=( 0.0320, 0.0129,-0.0088), 0 bond

In [60]:
box.visualize()

/home/chris/cme/mbuild/mbuild/compound.py:2443: UserWarning: Guessing that "<cc_r5 pos=( 0.1348, 0.0349, 0.0089), 0 bonds, id: 139868984105936>" is element: "C"
  atom, element))
/home/chris/cme/mbuild/mbuild/compound.py:2443: UserWarning: Guessing that "<ss pos=( 0.1231, 0.0224, 0.0088), 0 bonds, id: 139868984106320>" is element: "S"
  atom, element))
/home/chris/cme/mbuild/mbuild/compound.py:2443: UserWarning: Guessing that "<c3 pos=( 0.1603, 0.0386, 0.0089), 0 bonds, id: 139868984105168>" is element: "C"
  atom, element))
/home/chris/cme/mbuild/mbuild/compound.py:2443: UserWarning: Guessing that "<h4 pos=( 0.1314, 0.0452, 0.0088), 0 bonds, id: 139868982503760>" is element: "H"
  atom, element))
/home/chris/cme/mbuild/mbuild/compound.py:2443: UserWarning: Guessing that "<ha pos=( 0.1570, 0.0099, 0.0090), 0 bonds, id: 139868982501776>" is element: "H"
  atom, element))
/home/chris/cme/mbuild/mbuild/compound.py:2443: UserWarning: Guessing that "<hc pos=( 0.1663, 0.0361, 0.0001), 0 bond

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

/home/chris/cme/mbuild/mbuild/compound.py:2749: UserWarning: No unitcell detected for pybel.Molecule c1ccccc1	

  warn("No unitcell detected for pybel.Molecule {}".format(pybel_mol))


In [39]:
benzene = mb.load('c1ccccc1', smiles=True) # untyped mbuild compound
GAFF = foyer.forcefields.load_GAFF()
benzene_typed_pmd = GAFF.apply(benzene, assert_dihedral_params=False) # parmed structure
#benzene_typed_mb = mb.compound.Compound.from_parmed(structure=benzene_typed_pmd)
mb.compound.Compound.from_parmed(benzene, benzene_typed_pmd)


#benzene_pmd = benzene.to_parmed()
benzene_mass = np.sum([atom.mass for atom in benzene_typed.atoms]) # mass of compound calc'd from parmed struct
benzene.mass = benzene_mass # update mbuild compound mass

for atom_pmd, atom_mb in zip(benzene_typed, benzene):
#    print(atom_mb)
#    print(atom_pmd)
    atom_mb.name = "_{}".format(atom_pmd.type)
#    print(atom_mb.name)
#    print('---------')

[p.name for p in benzene]
box = mb.packing.fill_box(benzene, n_compounds=2, density=0.50)
box.visualize()

/home/chris/miniconda3/envs/mosdef-smiles/lib/python3.7/site-packages/openbabel/__init__.py:14: UserWarning: "import openbabel" is deprecated, instead use "from openbabel import openbabel"
  warnings.warn('"import openbabel" is deprecated, instead use "from openbabel import openbabel"')
/home/chris/cme/mbuild/mbuild/compound.py:2749: UserWarning: No unitcell detected for pybel.Molecule c1ccccc1	

  warn("No unitcell detected for pybel.Molecule {}".format(pybel_mol))
/home/chris/cme/forks/foyer/foyer/validator.py:132: ValidationWarning: You have empty smart definition(s)
  warn("You have empty smart definition(s)", ValidationWarning)
/home/chris/cme/mbuild/mbuild/compound.py:2443: UserWarning: Guessing that "<_ca pos=( 0.1377,-0.0219, 0.0004), 0 bonds, id: 140521374781520>" is element: "EP"
  atom, element))
/home/chris/cme/mbuild/mbuild/compound.py:2443: UserWarning: Guessing that "<_ha pos=( 0.2451,-0.0386, 0.0010), 0 bonds, id: 140521363608848>" is element: "EP"
  atom, element))
/ho

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol